In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
# from tqdm import tqdm_notebook as tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.cnn_trainer import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

output_dir = Path(r'C:\Users\admin\Google Drive\Datasets\CERT_output')
answers_dir = Path(r"C:\Users\admin\Google Drive\Datasets\CERT\answers")
main_answers_file = answers_dir / "insiders.csv"

lstm_checkpoint = output_dir / 'checkpoints/lstm/final-nll/final_model_3040.pth'
assert(lstm_checkpoint.is_file())

run_name = 'cnn/test'
log_dir = output_dir / 'logs' / run_name
checkpoint_dir = output_dir / 'checkpoints' / run_name

# assert(not log_dir.is_dir())
# assert(not checkpoint_dir.is_dir())

if log_dir.is_dir():
    shutil.rmtree(log_dir)
if checkpoint_dir.is_dir():
    shutil.rmtree(checkpoint_dir)

In [6]:
actions, targets = CertDataset.prepare_dataset(output_dir / 'aggregated.pkl', main_answers_file, min_length=50, max_length=200)

In [7]:
cert_dataset = CertDataset(actions, targets)
train_loader, val_loader = create_data_loaders(cert_dataset, validation_split=0.3, random_seed=0, batch_size=128)

params = get_params()

device = 'cuda'

In [11]:
model = InsiderClassifier(params['model'], lstm_checkpoint)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

train_engine = create_supervised_trainer(
                                        model, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch,
                                        log_dir=(output_dir / 'logs' / 'cnn' / run_name).as_posix(),
                                        checkpoint_dir=output_dir / 'checkpoints' / 'cnn' / run_name
                                       )

val_engine = create_supervised_evaluator(
        model, device=device,
        prepare_batch=prepare_batch,
        metrics={},
        criterion=criterion,
        log_dir=log_dir.as_posix(),
)

@train_engine.on(Events.STARTED)
def log_training_results(trainer):
    print('Initial validation run:')
    val_engine.train_epoch = 0
    val_engine.run(val_loader)

@train_engine.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    print('Validation run:')
    val_engine.train_epoch = train_engine.state.epoch
    val_engine.run(val_loader)


In [12]:
train_engine.run(train_loader, max_epochs=500)

Initial validation run:



Validation Results - Avg loss: 0.625840, Accuracy: 0.996334



Validation run:



Validation Results - Avg loss: 0.188772, Accuracy: 0.996334



Validation run:


...


Validation run:



Validation Results - Avg loss: 0.188772, Accuracy: 0.996334



Validation run:


Current run is terminating due to exception: .
Engine run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [10]:
%debug

> c:\anaconda3\envs\python3.6-torch\lib\site-packages\ignite\metrics\accuracy.py(49)_check_binary_multilabel_cases()
     47 
     48         if not torch.equal(y_pred, y_pred ** 2):
---> 49             raise ValueError("For binary cases, y_pred must be comprised of 0's and 1's.")
     50 
     51     def _check_type(self, output):



ipdb>  y_pred


tensor([0.5152, 0.5150, 0.5135, 0.5143, 0.5145, 0.5153, 0.5147, 0.5145, 0.5135,
        0.5136, 0.5135, 0.5136, 0.5146, 0.5142, 0.5156, 0.5142, 0.5186, 0.5135,
        0.5151, 0.5141, 0.5152, 0.5144, 0.5152, 0.5143, 0.5103, 0.5141, 0.5136,
        0.5135, 0.5132, 0.5145, 0.5140, 0.5160, 0.5136, 0.5138, 0.5152, 0.5144,
        0.5135, 0.5154, 0.5136, 0.5147, 0.5138, 0.5141, 0.5133, 0.5126, 0.5150,
        0.5135, 0.5151, 0.5135, 0.5135, 0.5138, 0.5138, 0.5135, 0.5197, 0.5145,
        0.5125, 0.5151, 0.5167, 0.5149, 0.5135, 0.5140, 0.5140, 0.5134, 0.5152,
        0.5149, 0.5135, 0.5151, 0.5169, 0.5137, 0.5135, 0.5158, 0.5146, 0.5135,
        0.5150, 0.5147, 0.5151, 0.5139, 0.5147, 0.5146, 0.5151, 0.5137, 0.5139,
        0.5142, 0.5142, 0.5187, 0.5140, 0.5136, 0.5139, 0.5138, 0.5107, 0.5137,
        0.5151, 0.5136, 0.5141, 0.5133, 0.5141, 0.5143, 0.5144, 0.5142, 0.5150,
        0.5134, 0.5144, 0.5148, 0.5134, 0.5151, 0.5147, 0.5153, 0.5143, 0.5137,
        0.5144, 0.5151, 0.5147, 0.5135, 

ipdb>  y


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0', dtype=torch.uint8)


ipdb>  exit
